<a href="https://colab.research.google.com/github/TirendazAcademy/Tracking-Agents-with-Comet/blob/main/tracking_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U openai langchain tiktoken comet_llm google-search-results cohere chromadb tensorflow-probability kaleido python-multipart

# Importing the necessary libraries

In [1]:
import os
import comet_llm

os.environ["COMET_API_KEY"] = "YOUR_API_KEY"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
os.environ["SERPAPI_API_KEY"] = "YOUR_API_KEY"

comet_llm.init(project="comet-langchain-test")

# Scraping the data

In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page to scrape
url = 'https://en.wikipedia.org/wiki/MrBeast'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the text on the page
text = soup.get_text()

# Open a new file called 'output.txt' in write mode and store the file object in a variable
with open('output.txt', 'w', encoding='utf-8') as file:
    # Write the string to the file
    file.write(text)

# load the document
with open('./output.txt', encoding='utf-8') as f:
    text = f.read()

# Data ingestion

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# define the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.create_documents([text])

# define the embeddings model
embeddings = OpenAIEmbeddings()

# use the text chunks and the embeddings model to fill our vector store
db = Chroma.from_documents(texts, embeddings)

# Tracking an agent with Comet ML

In [4]:
from langchain.llms import OpenAI
from langchain.callbacks.tracers.comet import CometTracer
from langchain.agents import initialize_agent, load_tools
from langchain import PromptTemplate

# Create a comet callback
comet_callback = CometTracer()
callbacks = [comet_callback]

# Define an openai llm
llm = OpenAI(temperature=0.9, callbacks=callbacks, verbose=True)

# Define a question
users_question = "What is MrBeast's age to the power of 0.54?"

# use our vector store to find similar text chunks
results = db.similarity_search(
    query=users_question,
    n_results=5
)

# define the prompt template
template = """

You are a chatbot who loves helping people! Given the context sections below, \
answer the question using only the context provided.

Context sections:
{context}

Question:
{users_question}

Answer:
"""

# Create a prompt template
prompt = PromptTemplate(template=template, input_variables=["context", "users_question"])

# Fill the prompt template
prompt_text = prompt.format(context = results, users_question = users_question)

# Load tools
tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=callbacks, verbose=True)

# Initialize an agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    callbacks=callbacks,
    verbose=True,
)

# Run the agent
agent.run(prompt_text, callbacks=callbacks,)
comet_callback.flush()



> Entering new AgentExecutor chain...
 I need to calculate the age of Mr Beast 
Action: Calculator 
Action Input: 25^0.54 
Observation: Answer: 5.687057308780144
Thought: I now know the final answer
Final Answer: 5.687057308780144
Chain logged to https://www.comet.com/tirendaz/comet-langchain-test


INFO:comet_llm.summary:Chain logged to https://www.comet.com/tirendaz/comet-langchain-test



> Finished chain.


## Resources

- [All You Need to Know to Build Your First LLM App](https://towardsdatascience.com/all-you-need-to-know-to-build-your-first-llm-app-eb982c78ffac)
- [Conversation QA Gradio ](https://github.com/hwchase17/conversation-qa-gradio/tree/master)

Thanks for reading. Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎